In [1]:
# import
import json
import time, datetime
from datetime import datetime
from datetime import timedelta

from bs4 import BeautifulSoup
import grequests
import requests
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 1000)

D:\develop\python\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
# 读取分类表
df_category = pd.read_excel('资产配置分类表.xlsx',dtype={'基金代码': np.object, '分类ID': np.int64})
# df_category.基金代码.unique()
# 基金名称	基金简称	基金代码	一级分类	二级分类	三级分类	分类ID	市场

In [3]:
# config & cookie

global_name = '康力泉'
# global_name = '李淑云'

klq_header = {'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'zh-CN',
            'Cache-Control': 'no-cache',
            'Connection': 'Keep-Alive',
            'Content-Length': '0',
            'Content-Type': 'application/json; charset=utf-8',
            'Cookie': 'st_si=14323438629411; st_sn=6; st_psi=20200902163818182-119085303933-6016543276; st_asi=delete; FundTradeLoginUser=g7FwsVi7RZ5iFbz/BIZ8IqVu4JhkfE8KU6k/cv/XETdjo2dqirERWbFEtFXChqZcRQCRciTA; TradeLoginToken=3a4154c219c143189c46ddfd2bacdfda; cp_token=40c434b1df0942c189f47352be6de1f0; st_pvi=12958943997933; st_sp=2020-09-01%2012%3A53%3A47; st_inirUrl=https%3A%2F%2Ffund.eastmoney.com%2F; fund_trade_cn=gx55YVIrqQPy96ZB9G/teoVQ3tsKZY6SxsrxaxhgRqkAMNOEG2JibeC3cPhZHzFnMnUzgM2fl9CBoePOmqndFVJkWifQuBkioXcQ5K+ld9Ln3fSsFsQ=; fund_trade_name=gQxvkpSKhZ8l++h2CqZ3ARXWojykOaO8scs3cNURWEJlu2dMlaW/L/FwPzh271xcgfjHaIu/; fund_trade_visitor=gHNHMiYbsZsqIyCh+NZRMlCi1nqksCuYz+hxcwbK3GTie2ks5VUnrtFsZtNp1F2c5wfNzGyv; fund_trade_risk=gS4aJpHdRZiEQFRGy7Zi59AP5ONkJjrjjf+ZcH2oJzA2T2fuG31kQtFLANwbMbAcP+M7D7Hx; fund_trade_gps=6; VipLevel=1; UTOKEN=gx55YVIrqQPy96ZB9G/teoVQ3tsKZY6SxsrxaxhgRqkAMNOEG2JibeC3cPhZHzFnMnUzg22VxnpCTreqnOcmFG7CGH3Xk9LyG1ccU033r+fC1NkgeL8=; LToken=77187386762f4eb7a15b81447ad15263; fund_trade_trackid=igaQrON5G3R5ayRIbKEWu3JkQ8QrKE5ivybo6advS3/2uurtmgX0dscf9G4qDZVIHOoekw3/Gxq5m5C4546M5w==; FundTradeLoginTab=0; FundTradeLoginCard=0; ASP.NET_SessionId=yaxrzpwkekm55paiqpozaznz',
            'Host': 'trade.1234567.com.cn',
            'Origin': 'https://trade.1234567.com.cn',
            'Referer': 'https://trade.1234567.com.cn/MyAssets/default',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
            'X-Requested-With': 'XMLHttpRequest'}
    
lsy_header = {'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'zh-CN',
            'Cache-Control': 'no-cache',
            'Connection': 'Keep-Alive',
            'Content-Length': '0',
            'Content-Type': 'application/json; charset=utf-8',
            'Cookie': 'st_si=14323438629411; st_sn=7; st_psi=20200902153037260-119085303933-2289424201; st_asi=delete; FundTradeLoginCard=0; FundTradeLoginUser=C+eXm5cGzMZS0egQYswQ5Vb/hCZEeT9OWf577NPlrmABN2NI0fifnpE0L22LphVTKj2WiAyN; FundTradeLoginTab=0; TradeLoginToken=552be43d0dcd48e09f1505bbb364e984; cp_token=9c4b8a105d10414eb5c0a5176b72149a; st_pvi=12958943997933; st_sp=2020-09-01%2012%3A53%3A47; st_inirUrl=https%3A%2F%2Ffund.eastmoney.com%2F; fund_trade_cn=CDPCw2ySJ8aAXvM+dWWqxan7NUvpw1kIW0lhQIRrgpERFtQztN4YazfM7N4AtWfO6QQoqU25zEfusjd42qtIEPof++7CYbGhSPTrtP0jDxlsp3E0IEg=; fund_trade_name=CxGU/VPlWMb9EZuXR/wVLip7aIREz0D5mi1K7l7DxGm8k25sc8r8PLEiEe3QHZzTzNSDfMHt; fund_trade_visitor=CBlheZC2DMvt4l/g+SwCVx1uro3E6Ypadkra7zzXrUi9Y2M7sunnDKE0RMZcn21TdzQpb4Cj; fund_trade_risk=Ce2YSZH49MeyARhhJsw0Om9YehyEyFDvnqWP7McTTKLJR2lS3xCFenEC1W2ykoqTfYAZDeF1; fund_trade_gps=7; VipLevel=1; UTOKEN=CDPCw2ySJ8aAXvM+dWWqxan7NUvpw1kIW0lhQIRrgpERFtQztN4YazfM7N4AtWfO6QQoqQ2KTPTOd+f0kXPyExZeSX+7s+OR5STsvY1RKweFoPmGeFs=; LToken=2446b234b9194626bb0313d5f8b895b1; fund_trade_trackid=CXtJxpfrWdmYtO8oWfgh+u0/mECcUxYT0tIn4KAE2ZNcmeHib116rYJHloYJvMnxWf9khDYYZZ4R/sm7fsJC5A==; ASP.NET_SessionId=q1sxnrip1n0rw2nx05lfzlpl',
            'Host': 'trade7.1234567.com.cn',
            'Origin': 'https://trade7.1234567.com.cn',
            'Referer': 'https://trade7.1234567.com.cn/MyAssets/default',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
            'X-Requested-With': 'XMLHttpRequest'}

headers = klq_header
# headers = lsy_header

# 日期函数

# 根据今天日期和起始日期,生成多个 90 天时间间隔的二维数组
def getDateIntervals(startYear = 2016, startMonth = 5, startDay = 1, interval = 90):
    dateIntervals = []
    # 起始日期为开户日
    startDate = datetime(year = startYear, month = startMonth, day = startDay)
    # 间隔 90 天
    interval_days = timedelta(days = interval)
    # 和起始日间隔 90 天后的日期
    endDate = startDate + interval_days
    # 终止日期为今天
    today = datetime.now()
    # 字符串格式化
    fmt = '%Y-%m-%d'
    while max(today, endDate) == today:
        # print(startDate.strftime(fmt), endDate.strftime(fmt))
        dateIntervals.append({'startDate': startDate.strftime(fmt), 'endDate': endDate.strftime(fmt), 'duration': interval_days.days})
        startDate = endDate + timedelta(days=1)
        endDate = startDate + interval_days
    # 最后一条不足 90 天的数据
    startDate = endDate - interval_days
    endDate = today
    # print(startDate.strftime(fmt), endDate.strftime(fmt))
    dateIntervals.append({'startDate': startDate.strftime(fmt), 'endDate': endDate.strftime(fmt), 'duration': (today - startDate).days})
    return dateIntervals

# 生成日期间隔
dateIntervals = getDateIntervals()
# dateIntervals = getDateIntervals(startYear = 2018, startMonth = 1, startDay = 1)
print(len(dateIntervals))
dateIntervals

18


[{'startDate': '2016-05-01', 'endDate': '2016-07-30', 'duration': 90},
 {'startDate': '2016-07-31', 'endDate': '2016-10-29', 'duration': 90},
 {'startDate': '2016-10-30', 'endDate': '2017-01-28', 'duration': 90},
 {'startDate': '2017-01-29', 'endDate': '2017-04-29', 'duration': 90},
 {'startDate': '2017-04-30', 'endDate': '2017-07-29', 'duration': 90},
 {'startDate': '2017-07-30', 'endDate': '2017-10-28', 'duration': 90},
 {'startDate': '2017-10-29', 'endDate': '2018-01-27', 'duration': 90},
 {'startDate': '2018-01-28', 'endDate': '2018-04-28', 'duration': 90},
 {'startDate': '2018-04-29', 'endDate': '2018-07-28', 'duration': 90},
 {'startDate': '2018-07-29', 'endDate': '2018-10-27', 'duration': 90},
 {'startDate': '2018-10-28', 'endDate': '2019-01-26', 'duration': 90},
 {'startDate': '2019-01-27', 'endDate': '2019-04-27', 'duration': 90},
 {'startDate': '2019-04-28', 'endDate': '2019-07-27', 'duration': 90},
 {'startDate': '2019-07-28', 'endDate': '2019-10-26', 'duration': 90},
 {'sta

In [4]:
# TODO 天天也有转托管 addition.json


In [5]:
# 请求所有 dateinterval 的数据

# 请求执行回调
def request_logging(r, *args, **kwargs):
    print('正在请求：{0}'.format(r.url))
    pass

# 错误处理
def err_handler(request, exception):
  print("{0} 请求出错: {1}".format(request.url, exception))

trade_list_url = u'https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate={0}&EndDate={1}&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1'
urls = []
# [print(x) for x in urls]
for interval in dateIntervals:
    urls.append(trade_list_url.format(interval['startDate'], interval['endDate']))
tasks = [grequests.get(x, headers=headers, callback=request_logging) for x in urls]
# 控制一下并发量，天天服务器没那么抗造
response_list = grequests.map(tasks, exception_handler=err_handler, size=2)
[print(x.request.url, x.status_code, x.text.strip()[0:30]) for x in response_list]

正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2016-05-01&EndDate=2016-07-30&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1
正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2016-07-31&EndDate=2016-10-29&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1
正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2016-10-30&EndDate=2017-01-28&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1
正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2017-01-29&EndDate=2017-04-29&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1
正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2017-04-30&EndDate=2017-07-29&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=1
正在请求：https://query.1234567.com.cn/Query/DelegateList?DataType=1&StartDate=2017-07-30&EndDate=2017-10-28&BusType=0&Statu=0&Account=&FundType=0&PageSize=500&PageIndex=

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
# response_list[0].text.strip()

In [7]:
# 整合所有的交易列表，生成详情 url 集合
def get_detail_urls_from_tradelist(htmlText):
    detail_url_list = []
    detail_url_prefix = u'https://query.1234567.com.cn'
    soup = BeautifulSoup(htmlText, 'lxml')
    results = soup.findAll(lambda e: e.name == 'a' and '详情' in e.text)
    # 接口拿回来默认是时间倒叙的
    results = list(reversed(results))
    [detail_url_list.append(detail_url_prefix + x['href']) for x in results]
    return detail_url_list

# 获取详细交易数据的整合信息
def get_detail_info(htmlText, url):
    soup = BeautifulSoup(htmlText, 'lxml')
#     print('url:' + url)
    # 从 ui-detail 中获取申请详细时间
    # 注意，分红信息是没有申请详细模块的，所以直接给 14:59:00
    ui_details = soup.findAll('div',{'class':'ui-detail'})
    if len(ui_details) > 0:
        applyTime = ui_details[0].select('td')[1].text.split(' ')[1]
        # 如果时间大于 15:00:00，如：18:39:44，则申请所在交易日应该是下一日
        # 但是天天基金给出的 applyDate 已经自动切换到申购交易日了，所以这些统一改成 14:59:00
        if int(applyTime[0:2]) >= 15:
            applyTime = '14:59:00'
    else:
        applyTime = '14:59:00'
        
    # 取申请、确认区
    results = soup.findAll('div',{'class':'ui-confirm'})
    applyInfo_keys = ['applyDate', 'applyOperate','applyStatus']
    confirmInfo_keys = ['fundName', 'fundCode', 'confirmDate', 'confirmOperate', 'confirmStatus', 'confirmNavUnit', 'confirmNavAcc', 'confirmMoney', 'confirmVolume', 'fee']

    jsonData = {'申请信息': {}, '确认信息': {}, '详情页': url }
    # 申请日期（拿累计净值）
    applyDate = ''
    for x in results:
        h3_array = x.select('h3')
        # 结果
        values = []
        if len(h3_array) > 0 and h3_array[0].text == u'申请信息':
            if len(x.select('table')) > 0:
                # 所有值
                tags = x.select('table')[0].tbody.findAll('td')
                # 不要“申请数额”
                tags.pop(-2)
                # 不要“标题<br/>代码”
                tags.pop(1)
                [values.append(x.text) for x in tags]
                jsonData['申请信息'] = (dict(zip(applyInfo_keys, values)))
                jsonData['申请信息']['applyTime'] = applyTime
        if len(h3_array) > 0 and h3_array[0].text == u'确认信息':
            if len(x.select('table')) > 0:
                # 所有值
                tags = x.select('table')[0].tbody.findAll('td')
                # “标题<br/>代码”
                a = tags.pop(1).a
                # 标题
                values.append(a.contents[0])
                # 代码
                values.append(a.contents[2])
                [values.append(x.text) for x in tags]
                # 拿一下累计净值
                values.insert(6, '待实现')
                jsonData['确认信息'] = (dict(zip(confirmInfo_keys, values)))
    return jsonData

# 遍历 trade_list
record_detail_urls = []
series_list = []
i = 1
for response in response_list:
    date_interval_urls = get_detail_urls_from_tradelist(response.text)
#     [record_detail_urls.append(x) for x in date_interval_urls]
    print('{0}/{1} total:{2}'.format(i, len(response_list) ,len(date_interval_urls)))
    i += 1
    tasks = [grequests.get(x, headers=headers) for x in date_interval_urls]
    # 控制一下并发量，天天服务器没那么抗造
    record_response_list = grequests.map(tasks, exception_handler=err_handler, size=2)
    for response in record_response_list:
        jsonData = get_detail_info(response.text, response.request.url)
#         print(jsonData)
#         print(response.request.url)
        item = dict()
        if len(jsonData['申请信息'].keys()) > 0:
            for k, v in jsonData['申请信息'].items():
                item[k] = v
        if len(jsonData['确认信息'].keys()) > 0:
            for k, v in jsonData['确认信息'].items():
                item[k] = v
        item['详情页'] = jsonData['详情页']
        series_list.append(pd.Series(item))
#     break
df_raw = pd.DataFrame(series_list)
df_raw = df_raw[['applyDate', 'applyTime', 'applyOperate', 'applyStatus', 'fundName', 'fundCode', 'confirmDate', 'confirmOperate', 'confirmStatus', 'confirmNavUnit', 'confirmNavAcc', 'confirmMoney', 'confirmVolume', 'fee', '详情页']]
df_raw.to_excel('01_{0}_record_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
df_raw

1/18 total:4
2/18 total:1
3/18 total:4
4/18 total:5
5/18 total:3
6/18 total:0
7/18 total:2
8/18 total:2
9/18 total:99
10/18 total:209
11/18 total:146
12/18 total:130
13/18 total:198
14/18 total:91
15/18 total:15
16/18 total:26
17/18 total:23
18/18 total:29


,applyDate,applyTime,applyOperate,applyStatus,fundName,fundCode,confirmDate,confirmOperate,confirmStatus,confirmNavUnit,confirmNavAcc,confirmMoney,confirmVolume,fee,详情页
0,2016-05-11,13:09:40,买入申请,已受理(支付完成),招商中证白酒指数分级,161725,2016-05-12,申购确认,成功,0.8820,待实现,500.00,566.33,0.50,https://query.1234567.com.cn/Query/Detail?id=f...
1,2016-05-13,08:44:39,买入申请,已受理(支付完成),广发美国房地产指数,000179,2016-05-17,申购确认,成功,1.2420,待实现,500.00,402.05,0.65,https://query.1234567.com.cn/Query/Detail?id=c...
2,2016-05-24,14:59:00,赎回申请,已受理(无需支付),广发美国房地产指数,000179,2016-05-26,赎回确认,成功,1.2270,待实现,490.85,402.05,2.47,https://query.1234567.com.cn/Query/Detail?id=2...
3,2016-05-27,14:08:10,赎回申请,已受理(无需支付),招商中证白酒指数分级,161725,2016-05-30,赎回确认,成功,0.8440,待实现,475.59,566.33,2.39,https://query.1234567.com.cn/Query/Detail?id=2...
4,2016-09-14,11:04:04,买入申请,已受理(支付完成),华安德国30(DAX)联接,000614,2016-09-20,申购确认,成功,1.0060,待实现,3400.00,3375.67,4.08,https://query.1234567.com.cn/Query/Detail?id=e...
5,2017-01-13,13:58:30,买入申请,已受理(支付完成),广发医药卫生联接A,001180,2017-01-16,申购确认,成功,0.7815,待实现,3700.00,4728.82,4.43,https://query.1234567.com.cn/Query/Detail?id=8...
6,2017-01-13,14:34:07,买入申请,已受理(支付完成),广发医药卫生联接A,001180,2017-01-16,申购确认,成功,0.7815,待实现,3700.00,4728.82,4.43,https://query.1234567.com.cn/Query/Detail?id=f...
7,2017-01-25,13:26:58,买入申请,已受理(支付完成),富国中证红利指数增强,100032,2017-01-26,申购确认,成功,1.1070,待实现,3960.00,3571.88,5.93,https://query.1234567.com.cn/Query/Detail?id=2...
8,2017-01-25,13:58:12,买入申请,已受理(支付完成),广发养老指数A,000968,2017-01-26,申购确认,成功,0.9595,待实现,3960.00,4122.20,4.75,https://query.1234567.com.cn/Query/Detail?id=0...
9,2017-02-15,10:16:01,买入申请,已受理(支付完成),广发中债7-10年国开债指数A,003376,2017-02-16,申购确认,成功,0.9504,待实现,3960.00,4164.58,1.98,https://query.1234567.com.cn/Query/Detail?id=1...


In [8]:
df_temp = df_raw.copy()

# df_temp = pd.read_excel('01_康力泉_record_list.xlsx',index_col=0)

# 	applyDate	applyTime	applyOperate	applyStatus	fundName	fundCode
# confirmDate	confirmOperate	confirmStatus	confirmNavUnit	confirmNavAcc	confirmMoney	confirmVolume	fee	

# applyOperate: '买入申请', nan, '充值申请', '赎回申请', '快速过户'
# applyStatus: '已受理(支付完成)', nan, '受理失败(支付失败)', '已受理(无需支付)', '已撤单(已支付)'
# confirmOperate: '申购确认', '赎回确认', '红利发放(红利再投资)', '转出投资账户确认', '转入投资账户确认', '转托管确认',
#                 '红利发放(现金分红)','强行调增'

# '转出投资账户确认', '转入投资账户确认' 是无意义的，属于网站行为

# 去掉已撤单 or 受理失败导致的记录，这是 fundName 为空仅有的合理解释
df_temp = df_temp[~(df_temp['fundName'].isnull())]
# 货币基金咱也不要了
df_temp = df_temp[~df_temp['fundName'].str.contains('货币')]
# '转出投资账户确认' & '转入投资账户确认' 无交易，只是组合操作，忽略, '转托管确认' 在 addition.json 中，此处忽略
df_temp = df_temp[~df_temp['confirmOperate'].isin(['转出投资账户确认', '转入投资账户确认', '转托管确认'])]
df_temp

def deal_type_calc(x):
    op = x
    if '申购' in op:
        return '买入'
    elif '赎回' in op:
        return '卖出'
    elif '红利' in op or '强行调' in op:
        return '分红'
    else:
        return x

def deal_money_calc(x):
    op = x['deal_type']
    money = float(x['confirmMoney'])
    fee = float(x['fee'])
    deal_money = 0.0
    if op == '买入':
        deal_money = round(money - fee, 2)
    elif op == '卖出':
        deal_money = money
    elif op == '分红':
        deal_money = money
    return deal_money

def occur_money_calc(x):
    op = x['deal_type']
    money = float(x['confirmMoney'])
    fee = float(x['fee'])
    occur_money = 0.0
    if op == '买入':
        occur_money = money
    elif op == '卖出':
        occur_money = round(money - fee, 2)
    elif op == '分红':
        occur_money = money
    return occur_money

def date_calc(x):
    if str(x['applyDate']) == 'nan':
        return x['confirmDate']
    else:
        return x['applyDate']

def time_calc(x):
    if str(x['applyTime']) == 'nan':
        return '14:59:00'
    else:
        return x['applyTime']
    
df_temp['id'] = df_temp.reset_index().index + 1
df_temp['date'] = df_temp.apply(date_calc, axis=1)
df_temp['time'] = df_temp.apply(time_calc, axis=1)
df_temp['code'] = df_temp['fundCode']
df_temp['name'] = df_temp['fundName']
df_temp['deal_type'] = df_temp['confirmOperate'].apply(deal_type_calc)
df_temp['volume'] = df_temp['confirmVolume']
# df_temp['fee'] = df_temp['fee']
df_temp['nav_unit'] = df_temp['confirmNavUnit']
df_temp['nav_acc'] = df_temp['confirmNavUnit']
df_temp['deal_money'] = df_temp.apply(deal_money_calc, axis=1)
df_temp['occur_money'] = df_temp.apply(occur_money_calc, axis=1)
df_temp['account'] = '{0}_天天'.format(global_name)
# # 补充一二三级分类
df_temp = pd.merge(df_temp, df_category, left_on='code', right_on='基金代码', how='left')
df_temp = df_temp.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
df_temp['note'] = df_temp['confirmOperate'] + '_' + df_temp['详情页']

df_temp = df_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
df_temp.to_excel('04_{0}_天天.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
df_temp

,id,date,time,code,name,deal_type,nav_unit,nav_acc,volume,deal_money,fee,occur_money,account,category1,category2,category3,category_id,note
0,1,2016-05-11,13:09:40,161725,招商中证白酒指数分级,买入,0.8820,0.8820,566.33,499.50,0.50,500.00,康力泉_天天,A 股,行业股,全指消费,148,申购确认_https://query.1234567.com.cn/Query/Detail...
1,2,2016-05-13,08:44:39,000179,广发美国房地产指数,买入,1.2420,1.2420,402.05,499.35,0.65,500.00,康力泉_天天,海外成熟,美国,房地产,314,申购确认_https://query.1234567.com.cn/Query/Detail...
2,3,2016-05-24,14:59:00,000179,广发美国房地产指数,卖出,1.2270,1.2270,402.05,490.85,2.47,488.38,康力泉_天天,海外成熟,美国,房地产,314,赎回确认_https://query.1234567.com.cn/Query/Detail...
3,4,2016-05-27,14:08:10,161725,招商中证白酒指数分级,卖出,0.8440,0.8440,566.33,475.59,2.39,473.20,康力泉_天天,A 股,行业股,全指消费,148,赎回确认_https://query.1234567.com.cn/Query/Detail...
4,5,2016-09-14,11:04:04,000614,华安德国30(DAX)联接,买入,1.0060,1.0060,3375.67,3395.92,4.08,3400.00,康力泉_天天,海外成熟,德国,德国30,321,申购确认_https://query.1234567.com.cn/Query/Detail...
5,6,2017-01-13,13:58:30,001180,广发医药卫生联接A,买入,0.7815,0.7815,4728.82,3695.57,4.43,3700.00,康力泉_天天,A 股,行业股,全指医药,142,申购确认_https://query.1234567.com.cn/Query/Detail...
6,7,2017-01-13,14:34:07,001180,广发医药卫生联接A,买入,0.7815,0.7815,4728.82,3695.57,4.43,3700.00,康力泉_天天,A 股,行业股,全指医药,142,申购确认_https://query.1234567.com.cn/Query/Detail...
7,8,2017-01-25,13:26:58,100032,富国中证红利指数增强,买入,1.1070,1.1070,3571.88,3954.07,5.93,3960.00,康力泉_天天,A 股,红利价值,中证红利,131,申购确认_https://query.1234567.com.cn/Query/Detail...
8,9,2017-01-25,13:58:12,000968,广发养老指数A,买入,0.9595,0.9595,4122.20,3955.25,4.75,3960.00,康力泉_天天,A 股,行业股,养老产业,141,申购确认_https://query.1234567.com.cn/Query/Detail...
9,10,2017-02-15,10:16:01,003376,广发中债7-10年国开债指数A,买入,0.9504,0.9504,4164.58,3958.02,1.98,3960.00,康力泉_天天,债券,国内债券,纯债,511,申购确认_https://query.1234567.com.cn/Query/Detail...
